In [3]:
import pandas as pd
import re
import os
import numpy as np
import datetime
from datetime import datetime as dt

In [4]:
csv_list = []

LUIGI_LOG_PATH = '/usr/local/trustyou/home/andream/nfs/processmining/pm4pyexample/'
#LUIGI_LOG_PATH = '/usr/local/trustyou/home/andream/nfs/processmining/minilogs/daily/logs/'
#LUIGI_LOG_PATH = '/usr/local/trustyou/home/andream/nfs/processmining/minilogs/table-precomp/logs/tch/'
#LUIGI_LOG_PATH='/usr/local/trustyou/home/andream/nfs/processmining/minilogs/table-precomp/logs/october/mongo/'
for filename in os.listdir(LUIGI_LOG_PATH):
    if filename.endswith('.csv'):
        log_path = LUIGI_LOG_PATH+'/'+filename
        csv_list.append(filename)
appended_df = pd.DataFrame()
#appended_df
csv_list.sort()
csv_list

['coma-running-example-just-two-cases.csv',
 'running-example-just-two-cases.csv',
 'running-example.csv',
 'semicolon-running-example-just-two-cases.csv']

In [5]:
from difflib import SequenceMatcher 
  
def longestSubstring(str1,str2): 
  
     # initialize SequenceMatcher object with  
     # input string 
     seqMatch = SequenceMatcher(None,str1,str2) 
  
     # find match of longest sub-string 
     # output will be like Match(a=0, b=0, size=5) 
     match = seqMatch.find_longest_match(0, len(str1), 0, len(str2)) 
  
     # print longest substring 
     if (match.size!=0): 
          result = str1[match.a: match.a + match.size]
          print (result)  
          return result
     else: 
          print ('No longest common sub-string found')

In [6]:
LUIGI_LOG_PATH = '/usr/local/trustyou/home/andream/nfs/processmining/pm4pyexample/'

print(os.listdir(LUIGI_LOG_PATH))

filename = 'running-example.csv'
if filename.endswith('.csv'):
        log_path = LUIGI_LOG_PATH+'/'+filename
        print('Preprocessing... ', filename)
        df = pd.read_csv(log_path, index_col=None, sep=';')
        df['timestamp'] = df[['dd-MM-yyyy:HH.mm']]
        df['activity'] = df[['Activity']]
        df['case'] = df[['Case ID']]
        df['state'] = 'start'
        df = df[['timestamp','activity','state', 'case']]
        print(len(df))
        
        df = df.reset_index()
        #appended_df = appended_df.append(df).reset_index()
        print(len(df))
df.head()

['running-example-just-two-cases.xes', 'running-example.csv', 'coma-running-example-just-two-cases.csv', 'just_two_cases_xes_im.png', 'just_two_cases_im.png', 'running-example-just-two-cases.csv', 'semicolon-running-example-just-two-cases.csv', 'just_two_cases_csv_im.png']
Preprocessing...  running-example.csv
42
42


,index,timestamp,activity,state,case
0,0,30-12-2010:11.02,register request,start,1
1,1,31-12-2010:10.06,examine thoroughly,start,1
2,2,05-01-2011:15.12,check ticket,start,1
3,3,06-01-2011:11.18,decide,start,1
4,4,07-01-2011:14.24,reject request,start,1


In [7]:
from datetime import datetime, timedelta

def add_duration_to_stamp(timestamp, duration):
    parsed_time = datetime.strptime(timestamp, '%d-%m-%Y:%H.%M')
    if duration.startswith('0'):
        parsed_duration = datetime.strptime(duration, "0-%H:%M:%S")
    else: 
        parsed_duration = datetime.strptime(duration, "%d-%H:%M:%S")
    then = parsed_time + timedelta(days=parsed_duration.day, 
                                   hours=parsed_duration.hour,
                                   minutes=parsed_duration.minute,
                                   seconds=parsed_duration.second)
    #days=parsed_duration.day,
                                   

    result = then.strftime('%d-%m-%Y:%H.%M')
    return result

In [8]:
#var = str(appended_df['index'][6])+'some'
#print(var)
#print(appended_df['index'])
appended_df = df.copy()
#appended_df = appended_df.sort_values(by='timestamp')
appended_df['timestamp'] = pd.to_datetime(appended_df['timestamp'], format='%d-%m-%Y:%H.%M').sort_values()
print(appended_df['timestamp'][0])
zero_timestamp = datetime.strptime(str(appended_df['timestamp'][0]), '%Y-%m-%d %H:%M:%S')
appended_df['timestamp'] = appended_df.apply(lambda row: (zero_timestamp+timedelta(days=row['index'])).strftime('%d-%m-%Y:%H.%M'), axis = 1)
appended_df.head()

2010-12-30 11:02:00


,index,timestamp,activity,state,case
0,0,30-12-2010:11.02,register request,start,1
1,1,31-12-2010:11.02,examine thoroughly,start,1
2,2,01-01-2011:11.02,check ticket,start,1
3,3,02-01-2011:11.02,decide,start,1
4,4,03-01-2011:11.02,reject request,start,1


In [9]:
end_df = appended_df.copy()
end_df['state'] = 'end'
#end_df['timestamp'] = end_df.apply(lambda row: row['timestamp'], axis=1)
end_df['timestamp'] = end_df.apply(lambda row: str(add_duration_to_stamp(row['timestamp'],'1-00:00:00')), axis=1)
#print(add_duration_to_stamp('30-12-2010:11.02','17:06:00'))
print(len(end_df))
end_df.head()

42


,index,timestamp,activity,state,case
0,0,31-12-2010:11.02,register request,end,1
1,1,01-01-2011:11.02,examine thoroughly,end,1
2,2,02-01-2011:11.02,check ticket,end,1
3,3,03-01-2011:11.02,decide,end,1
4,4,04-01-2011:11.02,reject request,end,1


In [10]:
appended_df = appended_df.append(end_df)
print(len(appended_df))
appended_df.tail()

84


,index,timestamp,activity,state,case
37,37,06-02-2011:11.02,register request,end,6
38,38,07-02-2011:11.02,examine casually,end,6
39,39,08-02-2011:11.02,check ticket,end,6
40,40,09-02-2011:11.02,decide,end,6
41,41,10-02-2011:11.02,pay compensation,end,6


In [11]:
df = appended_df
df = df[['activity','state','timestamp','case','index']]

df['start_time'] = np.where(df['state']=='start', df['timestamp'], np.nan)
df['end_time'] = np.where(df['state']=='end', df['timestamp'], np.nan)

start_df = df[df['state']=='start']
print('Start entries: ',len(start_df))

end_df = df[df['state']=='end']
print('End entries: ',len(end_df))

start_df.head()

Start entries:  42
End entries:  42


/usr/local/trustyou/home/andream/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/trustyou/home/andream/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,activity,state,timestamp,case,index,start_time,end_time
0,register request,start,30-12-2010:11.02,1,0,30-12-2010:11.02,NaN
1,examine thoroughly,start,31-12-2010:11.02,1,1,31-12-2010:11.02,NaN
2,check ticket,start,01-01-2011:11.02,1,2,01-01-2011:11.02,NaN
3,decide,start,02-01-2011:11.02,1,3,02-01-2011:11.02,NaN
4,reject request,start,03-01-2011:11.02,1,4,03-01-2011:11.02,NaN


In [12]:
def convertTimeFormat(date_str): 
    # To match format for pointTransformer.ipynb
    date_obj = dt.strptime(date_str, '%d-%m-%Y:%H.%M')
    result = dt.strftime(date_obj, '%Y-%m-%d %H:%M:%S')
    return result

In [13]:
tt = pd.merge(start_df, end_df, on=['activity', 'case', 'index'], how='inner')
tt = tt[tt['start_time_x'].notnull()]
tt = tt[tt['end_time_y'].notnull()]

tt['start_time'] = tt['start_time_x']
tt['end_time'] = tt['end_time_y']

tt['start_time'] = tt.apply(lambda row: convertTimeFormat(row['start_time']), axis=1)
tt['end_time'] = tt.apply(lambda row: convertTimeFormat(row['end_time']), axis=1)
tt['case'] = tt.apply(lambda row: str(row['case']), axis=1)


catt = tt[['case', 'activity','start_time','end_time']]
print(type(catt['case'][0]))
print(len(catt))
catt.head()

<class 'str'>
42


,case,activity,start_time,end_time
0,1,register request,2010-12-30 11:02:00,2010-12-31 11:02:00
1,1,examine thoroughly,2010-12-31 11:02:00,2011-01-01 11:02:00
2,1,check ticket,2011-01-01 11:02:00,2011-01-02 11:02:00
3,1,decide,2011-01-02 11:02:00,2011-01-03 11:02:00
4,1,reject request,2011-01-03 11:02:00,2011-01-04 11:02:00


In [26]:
counts = catt.groupby(['activity']).size().reset_index(name='counts').sort_values(by=['counts'], ascending=False)
counts = counts.sort_values(by=['counts'], ascending = False)
counts.head()

#appended = appended.head(100)
#len(appended)
unique_act = catt['activity'].unique().tolist()
print(len(unique_act), ' activities')
print(unique_act)

unique_trace = catt['case'].unique().tolist()
print(len(unique_trace), ' cases')

8  activities
['register request', 'examine thoroughly', 'check ticket', 'decide', 'reject request', 'examine casually', 'pay compensation', 'reinitiate request']
6  cases


In [27]:
catt_selection = catt[catt['case'].isin(['3'])]

unique_act = catt_selection['activity'].unique().tolist()
print(len(unique_act), ' activities')
print(unique_act)

unique_trace = catt_selection['case'].unique().tolist()
print(len(unique_trace), ' cases')
catt_selection

7  activities
['register request', 'examine casually', 'check ticket', 'decide', 'reinitiate request', 'examine thoroughly', 'pay compensation']
1  cases


,case,activity,start_time,end_time
10,3,register request,2011-01-09 11:02:00,2011-01-10 11:02:00
11,3,examine casually,2011-01-10 11:02:00,2011-01-11 11:02:00
12,3,check ticket,2011-01-11 11:02:00,2011-01-12 11:02:00
13,3,decide,2011-01-12 11:02:00,2011-01-13 11:02:00
14,3,reinitiate request,2011-01-13 11:02:00,2011-01-14 11:02:00
15,3,examine thoroughly,2011-01-14 11:02:00,2011-01-15 11:02:00
16,3,check ticket,2011-01-15 11:02:00,2011-01-16 11:02:00
17,3,decide,2011-01-16 11:02:00,2011-01-17 11:02:00
18,3,pay compensation,2011-01-17 11:02:00,2011-01-18 11:02:00


In [28]:
OUTPUT_CATT_PATH = LUIGI_LOG_PATH+'/../experiments/catts_1trace_7act_1dayDur_meets_'+filename
print(len(catt))
print(OUTPUT_CATT_PATH)
catt_selection.to_csv(OUTPUT_CATT_PATH, index = None, header=True)

42
/usr/local/trustyou/home/andream/nfs/processmining/pm4pyexample//../experiments/catts_1trace_7act_1dayDur_meets_running-example.csv
